**This notebook builds and validates a very simple neural network classifier.  The features and validation will be the same as the gbm model (income_model_binary.ipynb) in the gradient boosting repo.  Like that model, this is meant as a beginner's walkthrough.  Therefore it will have more explanation and less sophisitication than the complex models used for most complex applications.  With that being said, detailed explanations are not always provided in places where high quality materials can be found easily on the web.**

**Some comments may refer back to the gbm example so it would be good to view that tutorial first.  Subsequent neural network examples will be independent of the gbm example due to limitations with this data set.  However, it might be a good frame of reference for people making the transition to neural networks.  In fact, several steps are done in an inefficient manner for the sake of being compatible with that tutorial and to faciliate learning.  This will not be the case for other neural net applications.**

**First it is necessary to load the following packages.**

In [0]:
import pandas as pd
import numpy as np
from sklearn import ensemble
from sklearn import datasets
from sklearn.utils import shuffle
#from sklearn.metrics import mean_squared_error
from sklearn.metrics import matthews_corrcoef
#from sklearn.ensemble.partial_dependence import plot_partial_dependence
#from sklearn.ensemble.partial_dependence import partial_dependence
from sklearn.model_selection import train_test_split
#from sklearn.grid_search import GridSearchCV
#from decimal import Decimal
import torch
from torch import nn
#import torch.nn.functional as F
#from torchvision import datasets, transforms
from torch import optim
import torch.utils.data



**A list of column names is constructed and applied as the data is being read.  Pandas data frames are not optimal for most large data sets for which neural nets are used.  However, this is fine for the small data set being used in this tutorial.**

In [0]:

cols = ['age', 'workclass', 'fnlwgt', 'education', 'education_num',
        'marital_status', 'occupation', 'relationship', 'ethnicity',
        'gender', 'capital_gain', 'capital_loss', 'hours_per_week',
        'country_of_origin', 'income']

raw_data = pd.read_csv('http://archive.ics.uci.edu/ml/machine-learning-databases/adult/adult.data',
                       names = cols)

**As long as the data is not too large, it is a good idea (especially for beginners) to make a copy of the raw data.  This will allow easy checking of the before and after state of  the data after different manipulations are performed.  If this were a production pipeline, this would probably not be done.  Furthermore, copying very large data sets is inefficient and impractical.  This step remains only to maintain consistency with the boosted tree model previously referenced.**

In [0]:
data = raw_data


**Now check that the data looks as expected at this point.**

In [0]:
data.head()


,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,ethnicity,gender,capital_gain,capital_loss,hours_per_week,country_of_origin,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [0]:
data.columns


Index(['age', 'workclass', 'fnlwgt', 'education', 'education_num',
       'marital_status', 'occupation', 'relationship', 'ethnicity', 'gender',
       'capital_gain', 'capital_loss', 'hours_per_week', 'country_of_origin',
       'income'],
      dtype='object')

**The target must be defined.  This is a binary classification problem so it will be coded as 1/0.  The model itself will output a probability (between 0 and 1) when applied to data.  At that point, a method will be needed for transforming that probability into a firm prediction if the application calls for it.  For some applications (like credit scoring), the raw output is used in a different way.**

**First check the variable to be transformed**

In [0]:
data.income.value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

**There are two classes so this should be a straightforward conversion.**

In [0]:
target = np.zeros(len(data))

for i in range(len(data)):
    if data.income.values[i] == ' >50K':
        target[i] = 1
        
data['target'] = target


**Check that the conversion worked properly,**

In [0]:
data.target.value_counts()

0.0    24720
1.0     7841
Name: target, dtype: int64

In [0]:
data.income.value_counts()

 <=50K    24720
 >50K      7841
Name: income, dtype: int64

**The target is set.  Now it is time to do some feature engineering.**

**The gender column currently contains text.  It will need to undergo the same transformation as our target variable.  It will be coded as as 1 for female and 0 for male.**

In [0]:
gender_num = np.zeros(len(data))

for i in range(len(data)):
    if data.gender.values[i] == ' Female':
        gender_num[i] = 1
        
data['gender_num'] = gender_num

In [0]:
data.gender.value_counts()

 Male      21790
 Female    10771
Name: gender, dtype: int64

In [0]:
data.gender_num.value_counts()

0.0    21790
1.0    10771
Name: gender_num, dtype: int64


**The workclass column has text padded with white space.  It will make life easier if we strip out the wihite space.  We will also rename the classes because they contain symbols that will not work as column names. Some classes are collapsed as they are similar.  In this case, the assumption is that information is not lost by collapsing and there is increased predictive power in unifying these categories.  This is not always the case.  Careful thought should be given to collapsing categorical features and testing of different collapsing schema should be considered.**

In [0]:
data.workclass.value_counts()

 Private             22696
 Self-emp-not-inc     2541
 Local-gov            2093
 ?                    1836
 State-gov            1298
 Self-emp-inc         1116
 Federal-gov           960
 Without-pay            14
 Never-worked            7
Name: workclass, dtype: int64

In [0]:
data.workclass = data.workclass.str.strip()

data['workclass_buckets'] = data.workclass.replace(['Private' #begin old names
                                                    , 'Self-emp-not-inc'
                                                    , 'Local-gov'
                                                    , '?'
                                                    , 'State-gov'
                                                    , 'Self-emp-inc'
                                                    , 'Federal-gov'
                                                    , 'Without-pay'
                                                    , 'Never-worked'] 
                                                    , ['pvt' #begin new names
                                                    , 'self'
                                                    , 'govt'
                                                    , 'other'
                                                    , 'govt'
                                                    , 'self'
                                                    , 'govt'
                                                    , 'other'
                                                    , 'other'])

**Check that workclass collapsed properly.**

In [0]:
data.workclass_buckets.value_counts()

pvt      22696
govt      4351
self      3657
other     1857
Name: workclass_buckets, dtype: int64

**Since these features are purely categorical (no meaningful numeric assignment can be made), they will need to be one hot encoded.**

In [0]:
wcb = data.workclass_buckets
data = data.assign(wcb=wcb.values)
data.loc[:,'wcb'] = wcb.values
data = pd.get_dummies(data, columns=['wcb'], dtype = float)

**Check that the encoding was performed properly.**

In [0]:
data[['workclass', 'workclass_buckets',  'wcb_govt', 'wcb_self', 'wcb_pvt', 'wcb_other']].head(10)

,workclass,workclass_buckets,wcb_govt,wcb_self,wcb_pvt,wcb_other
0,State-gov,govt,1,0,0,0
1,Self-emp-not-inc,self,0,1,0,0
2,Private,pvt,0,0,1,0
3,Private,pvt,0,0,1,0
4,Private,pvt,0,0,1,0
5,Private,pvt,0,0,1,0
6,Private,pvt,0,0,1,0
7,Self-emp-not-inc,self,0,1,0,0
8,Private,pvt,0,0,1,0
9,Private,pvt,0,0,1,0


**The same process would need to be performed to include ethnicity.  A good exercise for beginners would be to go through the process of one hot encoding ethnicity and adding it to the model.**

**Now education_num will be examined.  There is also an education field which is categorical.  However, if the education_num values have numerical meaning, this would likely be a more powerful predictor.  A quick check will reveal if larger values of education_num are indicative of higher levels of education.**

In [0]:

data[['education_num', 'education']].groupby(['education_num', 'education']).count()

,
education_num,education
1,Preschool
2,1st-4th
3,5th-6th
4,7th-8th
5,9th
6,10th
7,11th
8,12th
9,HS-grad


**This result yields two important findings.  Firstly, education_num has a one to one mapping with education.  This was expected.  But as with all things data, it was still worth checking.  Also, it is pretty clear that the education_num values become larger as education level increases.  This means that the numerical values will be useful for modeling.**

**At this point feature engineering will stop and data will be split into a train and test set.  For this model, a 75%/25% split was chosen.  This is somewhat arbitrary for this application.  For other applications, there can be practical reasons why the data may need to be split differently.  In the absence of those types of constraints, most people will use something between 60%/40% and 80%/20%.  It is also worth noting that there is certainly opportunity to build more sophisticated features with this data set.  The feature engineering step is an opportunity to be creative and try new things.  In this case, some basic features were built for demonstration purposes.**

In [0]:
train, test = train_test_split(data, test_size = 0.25)

In [0]:
train.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,ethnicity,gender,...,hours_per_week,country_of_origin,income,target,gender_num,workclass_buckets,wcb_govt,wcb_other,wcb_pvt,wcb_self
6272,31,Private,137814,Some-college,10,Divorced,Sales,Own-child,White,Female,...,30,United-States,<=50K,0.0,1.0,pvt,0,0,1,0
4032,40,Private,184105,Assoc-voc,11,Married-civ-spouse,Craft-repair,Husband,White,Male,...,40,United-States,>50K,1.0,0.0,pvt,0,0,1,0
28622,57,Private,201991,Bachelors,13,Married-civ-spouse,Prof-specialty,Husband,White,Male,...,40,United-States,<=50K,0.0,0.0,pvt,0,0,1,0
9950,23,Private,175424,Some-college,10,Never-married,Adm-clerical,Own-child,Black,Female,...,40,United-States,<=50K,0.0,1.0,pvt,0,0,1,0
26569,27,Private,333296,Bachelors,13,Never-married,Other-service,Not-in-family,White,Male,...,30,?,<=50K,0.0,0.0,pvt,0,0,1,0


In [0]:
test.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,ethnicity,gender,...,hours_per_week,country_of_origin,income,target,gender_num,workclass_buckets,wcb_govt,wcb_other,wcb_pvt,wcb_self
32465,66,Private,269665,HS-grad,9,Widowed,Exec-managerial,Not-in-family,White,Female,...,25,United-States,<=50K,0.0,1.0,pvt,0,0,1,0
27764,38,Self-emp-not-inc,152621,Bachelors,13,Married-civ-spouse,Farming-fishing,Husband,White,Male,...,99,United-States,<=50K,0.0,0.0,self,0,0,0,1
29094,36,?,224886,HS-grad,9,Married-civ-spouse,?,Husband,White,Male,...,40,United-States,<=50K,0.0,0.0,other,0,1,0,0
8198,39,Federal-gov,200968,Some-college,10,Married-civ-spouse,Adm-clerical,Other-relative,White,Male,...,45,United-States,>50K,1.0,0.0,govt,1,0,0,0
25207,28,Private,188236,Some-college,10,Never-married,Adm-clerical,Not-in-family,White,Female,...,40,United-States,<=50K,0.0,1.0,pvt,0,0,1,0


**A list of features is assembled to control the inputs to the model.  The list is used to subset down to only the columns that will be used as features.  Editing this list will alter which features are used as inputs to the model.  For now, the features previously examined/constructed will be used as well as a few other columns that seem like reasonable predictors.**

In [0]:
features = ['wcb_govt'
            , 'wcb_self'
            , 'wcb_pvt'
            , 'wcb_other'
            , 'education_num'
            , 'age'
            , 'gender_num'
            , 'capital_gain'
            , 'capital_loss'
            , 'hours_per_week']


**Now the data will be split again.  The built in algorithm wants the features (X) and the target (y) in separate arrays.  Both the train and test data will be split in this way so they can be input into the function.**

In [0]:
X_train = np.array(train[features])
X_test = np.array(test[features])
y_train = np.array(train['target'])
y_test = np.array(test['target'])

**Now the data will need to be converted to torch tensors to feed the neural net.**

In [0]:
X_train = torch.from_numpy(X_train)
X_test = torch.from_numpy(X_test)
y_train = torch.from_numpy(y_train)
y_test = torch.from_numpy(y_test)

**A few more codes will get the data compatible with what the algorithm is expecting.  The neural net wants float datatypes and the target vector to have two dimensions.**

In [0]:
X_train = X_train.float()
y_train = y_train.float()
y_train = y_train.view(len(y_train), 1)

X_test = X_test.float()



**This network will have one hidden layer.  The number of input nodes will be the number of features.  The hidden layer will have a configurable number of nodes.  The output layer will have one node which will give the probabilty of income over $50k**

In [0]:
n_inputs = len(features)
n_hidden = 3
n_output = 1

**The architecture is designed below.  The hidden layer has a hyperbolic tangent activation function.  The nn.tanh() command is how this is defined.  Other activation functions are available.  A sigmoid activation function (nn.Sigmoid) will be used for the output layer so that a number between zero and one is produced that can be interpreted as a probability.**

**Rectified linear units (nn.ReLU) are also very popular as they are known to speed up training.  For this application, it probably does not make much of a difference. **

In [0]:
model = nn.Sequential(nn.Linear(n_inputs, n_hidden),
                      nn.Tanh(),
                      nn.Linear(n_hidden, n_output),
                      nn.Sigmoid())

**Now, a loss function must be chosen.  Binary cross entropy makes sense for this application.  Loss (or cost) functions are well outside the scope of this treatment.  However, information can easily be found on the web for those that are curious.**

In [0]:
criterion = nn.BCELoss()

**The next command tells the net to optimize using stochastic gradient descent with a configurable learning rate.  It has been set to .05 which is very fast.  However, this is such a simple model that anything slower will lead to overfitting.  More complex nets will need to be slowed down considerably.**

In [0]:
learning_rate = .05
optimizer = optim.SGD(model.parameters(), lr=learning_rate)

**The number of epochs will be left as configurable and the net is trained.**

In [0]:
epochs = 5
for e in range(epochs):

    # Training pass
    optimizer.zero_grad()

    output = model(X_train)
    loss = criterion(output, y_train)
    loss.backward()
    optimizer.step()

    loss = loss.item()

    print(f"Training loss: {loss}")

Training loss: 0.6795438528060913
Training loss: 0.5992354154586792
Training loss: 0.5806587934494019
Training loss: 0.5721477270126343
Training loss: 0.56684410572052


**Now some more data manipulation is performed and assignments are made based on the probabilities which were outputted by the sigmoid function.**

In [0]:
y_prob_train = output.detach().numpy()
y_train = y_train.detach().numpy()

In [0]:
y_pred_train = np.zeros(len(y_prob_train))

for i in range(len(y_prob_train)):
    if y_prob_train[i] >= .5:
        y_pred_train[i] = 1
        


**Now predictions for the test set are calculated.  The model is set to train mode by default so it is switched to evaluation model (model.eval()).  In this case, it is not really necessary.  However, with more advanced functionality it would be.**


In [0]:
model.eval()

Sequential(
  (0): Linear(in_features=10, out_features=3, bias=True)
  (1): Tanh()
  (2): Linear(in_features=3, out_features=1, bias=True)
  (3): Sigmoid()
)

In [0]:
with torch.no_grad():
    y_prob_test = model(X_test)

In [0]:
y_prob_test = y_prob_test.detach().numpy()

y_pred_test = np.zeros(len(y_prob_test))

for i in range(len(y_prob_test)):
    if y_prob_test[i] >= .5:
        y_pred_test[i] = 1
        


**Training set validation.**

In [0]:
matthews_corrcoef(y_train, y_pred_train)

0.21443646717066892

**Testing Set Validation.**

In [0]:
matthews_corrcoef(y_test, y_pred_test)

0.24645150782900976

**When trained and validated, this model has consistently produced MCCs of around .2 for both data sets.  These results are inferior to the boosted tree results.  There is no doubt that this model could be improved.  However, it took considerable tinkering just to get it to this point and would take a lot more to make it better.**

**For models with few features and low complexity, neural networks are not the best option.  The work it takes to tune them is really not worth it.  There is no guarantee it will be any better than the boosted tree model anyway.  However, larger and more complex data sets will demand the neural net approach.  This will be explored in other notebooks.**